In [ ]:
import requests
import pandas as pd


In [ ]:
from bs4 import BeautifulSoup
from urllib.parse import unquote

In [ ]:
url="https://med.stanford.edu/neurology/faculty/overview.html?tab=proxy"
res=requests.get(url)

In [ ]:
content=BeautifulSoup(res.content)

In [ ]:
rows = content.find_all("div", class_="panel_builder parbase section")
faculty_data = []

for row in rows:
    details = row.find_all('p')

     comma_index = detail.text.find(",")
        newline_index = detail.text.find("\n")
        index = comma_index if comma_index != -1 else newline_index
        name = detail.text[0:index]


    for detail in details:
        link_a = detail.find("a")
        profile_emails = []
        profile_links = []

        if link_a:
            link = link_a['href']
            res_profile = requests.get(link)
            content_profile = BeautifulSoup(res_profile.content)
            contact = content_profile.find("div", class_="contact sidebar-block")

            if contact:
                emails = contact.find_all("a")
                for email in emails:
                    personal_email = email.text
                    profile_emails.append(personal_email)

            links = content_profile.find("div", class_="links sidebar-block")
            if links:
                websites = links.find_all('a')
                for website in websites:
                    current_website = unquote(website['href'])

                    profile_links.append(current_website)

        comma_index = detail.text.find(",")
        newline_index = detail.text.find("\n")
        index = comma_index if comma_index != -1 else newline_index
        name = detail.text[0:index]
        faculty_data.append([name, ', '.join(profile_emails),  '/n'.join(profile_links)])

In [ ]:
df = pd.DataFrame(faculty_data, columns=['Name','Email','Links'])

In [ ]:
df.to_csv('faculty_data.csv')
df.to_excel('faculty_data.xlsx',index=False)